In [ ]:
import json
import random

# Constants
ITEM_TYPES = {
    "consumable": "consumable",
    "weapon": "weapon"
}


class Base:
    def __init__(self, name):
        self.name = name
        self.damage = 100
        self.hp = 100
        self.inventory = []

    def apply_item(self, item):
        if item["HP"] != 0:
            self.hp += item["HP"]
            print(f"{self.name}'s HP {'boosted' if item['HP'] > 0 else 'decreased'} by {abs(item['HP'])}.")

        if item["damage"] != 0:
            self.damage += item["damage"]
            print(f"{self.name}'s damage {'boosted' if item['damage'] > 0 else 'decreased'} by {abs(item['damage'])}.")

    def use_consumable(self, consumable_name):
        for item in self.inventory:
            if item["name"].lower() == consumable_name.lower() and item["Type"].lower() == ITEM_TYPES["consumable"]:
                self.apply_item(item)
                self.inventory.remove(item)
                print(f"{self.name} used {item['name']}.")
                return
        print(f"{consumable_name} not found in inventory.")

    def access_inventory(self, for_consumables=False):
        print("Inventory:")
        if for_consumables:
            items = [item for item in self.inventory if item["Type"].lower() == ITEM_TYPES["consumable"]]
        else:
            items = self.inventory
        display_items(items)

    def attributes(self):
        print(f"Welcome {self.name}! Choose your attributes:")


class Character(Base):
    def __init__(self, name):
        super().__init__(name)
        self.equipped_categories = {"footwear": False, "leg armor": False, "chest plate": False, "headgear": False,
                                    "weapon": False}

    def choose_attributes(self):
        super().attributes()
        self.hp += 10

    def choose_weapon(self, weapon_name):
        for item in self.inventory:
            if item["name"].lower() == weapon_name.lower() and item["Type"].lower() == ITEM_TYPES["weapon"]:
                self.apply_item(item)
                self.inventory.remove(item)
                self.equipped_categories["weapon"] = True
                print(f"{self.name} equipped {item['name']}.")
                return
        print(f"No weapon with name '{weapon_name}' found in inventory.")

    def equip_item(self, item_type, item_name): #User can equip an item from the given inventory
        while True:
            for item in self.inventory:
                if item["name"].lower()==item_name.lower() and item["Type"].lower()==item_type.lower():
                    self.apply_item(item)
                    self.inventory.remove(item)
                    self.equipped_categories[item_type.lower()] = True
                    print(f"{self.name} equipped {item['name']}.")
                    return  # Exits the loop after equipping an item
            print(f"{item_type} with name '{item_name}' not found in inventory. Please choose from the available items.")
            self.access_inventory()  # Shows available items in the inventory
            item_name=input(f"Enter the name of the {item_type} to equip: ")
        
    def all_categories_equipped(self):
        return all(self.equipped_categories.values())


def choose_character():
    characters = {
        "1": "Jefe",
        "2": "Mafins",
        "3": "Manchis",
        "4": "Tipiti",
        "5": "Panzon",
        "6": "Lagartijo"
    }
    
    print("Choose a character:")
    for num, char_name in characters.items():
        print(f"{num}. {char_name}")
    
    while True:
        choice = input("Enter the number corresponding to your choice: ")
        if choice in characters:
            return Character(characters[choice])
        else:
            print("Invalid choice. Please enter a valid number.")



def load_inventory(file_path="ItemValues.json"):
    try:
        with open(file_path, "r") as f:
            return json.load(f).get("items", [])
    except FileNotFoundError:
        print(f"Inventory file '{file_path}' not found.")
        return []


def display_items(items):
    print("--------------------------------------------------------------------")
    print("| {:<20} | {:<10} | {:<20} | {:<10} | {:<10} |".format("Name", "Type", "Description", "Damage", "HP"))
    print("--------------------------------------------------------------------")
    for item in items:
        print("| {:<20} | {:<10} | {:<20} | {:<10} | {:<10} |".format(item["name"], item["Type"], item["description"], item["damage"], item["HP"]))
    print("--------------------------------------------------------------------")


def battle(player, opponent):
    print(f"{player.name} is battling {opponent.name}!")
    while player.hp > 0 and opponent.hp > 0:
        print("\nNext Round:")
        print(f"Player HP: {player.hp}, Opponent HP: {opponent.hp}")
        player_attack = random.randint(1, player.damage)
        opponent_attack = random.randint(1, opponent.damage)
        print(f"{player.name} attacks {opponent.name} for {player_attack} damage!")
        opponent.hp -= player_attack
        if opponent.hp <= 0:
            print(f"{opponent.name} has been defeated!")
            break
        print(f"{opponent.name} attacks {player.name} for {opponent_attack} damage!")
        player.hp -= opponent_attack
        if player.hp <= 0:
            print(f"{player.name} has been defeated!")
            break
        choice = input("\nDo you want to (i)inventory, (f)ight, or (c)consume using a consumable? (i/f/c): ")
        if choice.lower() == "i":
            player.access_inventory()
            category = input("Enter the category to change the item (footwear, leg armor, chest plate, headgear, weapon): ")
            item_name = input(f"Enter the name of the {category} to equip: ")
            player.equip_item(category, item_name)
        elif choice.lower() == "c":
            player.access_inventory(for_consumables=True)
            consumable_name = input("Enter the name of the consumable to use: ")
            player.use_consumable(consumable_name)
            print("\nUpdated character stats:")
            print(f"Player HP: {player.hp}, Enemy HP: {opponent.hp}")
        elif choice.lower() == "f":
            continue
        else:
            print("Invalid choice. Continuing to fight.")

    else:
        print("The battle is over!")


def main():
    selected_character = choose_character()
    selected_character.choose_attributes()

    inventory = load_inventory()
    selected_character.inventory = inventory

    for category in selected_character.equipped_categories.keys():
        print(f"\nChoose {category}:")
        category_items = [item for item in inventory if item["Type"].lower() == category.lower()]
        display_items(category_items)
        
        while not selected_character.equipped_categories[category]:
            item_name = input(f"Enter the name of the {category.lower()} to equip (or type 'skip' to skip): ")
            if item_name.lower() == "skip":
                break
            selected_character.equip_item(category, item_name)

    print("\nItems equipped. Starting the fight...\n")

    opponent = Character("Lolo the Man-Eating Bunny")
    opponent.hp = 500
    opponent.damage = 100

    battle(selected_character, opponent)


if __name__ == "__main__":
    main()
